In [1]:
import os
# import tensorflow.compat.v2 as tf
import tensorflow as tf
import json
from tensorflow.keras import datasets, layers, models, metrics
from minio import Minio
from minio.error import ResponseError

In [2]:
minioClient = Minio('minio-service.kubeflow:9000',
              access_key='minio', secret_key='minio123', secure=False)
try:
    data = minioClient.get_object('dataset', 'mnist/mnist.npz')
    with open('/tmp/mnist.npz', 'wb') as file_data:
        for d in data.stream(32*1024):
            file_data.write(d)
except ResponseError as err:
    print(err)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path='/tmp/mnist.npz')
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
dropout_rate = 0.3
learning_rate = 0.1

In [4]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

sgd = tf.keras.optimizers.SGD(lr=learning_rate,
                                  decay=1e-6,
                                  momentum=0.9,
                                  nesterov=True)
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3)
score = model.evaluate(x_test,  y_test, verbose=2)
loss = score[0]
accuracy = score[1]

Train on 60000 samples
Epoch 1/3
60000/60000 [==============================] - 7s 119us/sample - loss: 0.4195 - accuracy: 0.8762
Epoch 2/3
60000/60000 [==============================] - 11s 177us/sample - loss: 0.3359 - accuracy: 0.9095
Epoch 3/3
60000/60000 [==============================] - 11s 180us/sample - loss: 0.3022 - accuracy: 0.9205
10000/10000 - 1s - loss: 0.1851 - accuracy: 0.9510


In [5]:
os.environ.update({
    'S3_ENDPOINT'          : 'minio-service.kubeflow:9000',
    'AWS_ACCESS_KEY_ID'    : 'minio',
    'AWS_SECRET_ACCESS_KEY': 'minio123',
    'S3_USE_HTTPS'         : '0',	# Whether or not to use HTTPS. Disable with 0.                        
    'S3_VERIFY_SSL'        : '0' 	# If HTTPS is used, controls if SSL should be enabled. Disable with 0.
})    
model.save("s3://model/mnist-model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: s3://model/mnist-model/assets


In [6]:
print(loss)
print(accuracy)

0.18509579109735788
0.951
